In [61]:
import tensorflow as tf
from pathlib import Path

In [62]:
import os
os.chdir('../DS_Projects/DeepCNN_Classifier_End_To_End/')

In [63]:
os.getcwd()

'c:\\DS_Projects\\DeepCNN_Classifier_End_To_End'

In [64]:
from dataclasses import dataclass

In [65]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    path : Path
    training_data : Path
    params_img_size : list
    params_batch_size : int

In [66]:
from DeepCNNClassifer.constants import *
from DeepCNNClassifer.utils import read_yaml, create_directories

In [67]:
class ConfigurationManager:
    def __init__(
            self,
            config_path = CONFIG_FILE_PATH,
            params_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifacts_root])
        
    def get_evaluation_config(self) -> ModelEvaluationConfig:
        model_evaluation_config = ModelEvaluationConfig(
            path = 'artifacts/training/model.h5',
            training_data= 'artifacts/data_ingestion/PetImages',
            params_img_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
        )
        return model_evaluation_config

In [68]:
from DeepCNNClassifer.utils import *

In [69]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    def _valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
                target_size = self.config.params_img_size[:-1],
                batch_size = self.config.params_batch_size,
                interpolation = "bilinear"
            )
        
        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_data_generator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle = False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path : Path):
        return tf.keras.models.load_model(path)

    def evaluate(self):
        model = self.load_model(self.config.path)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_scores(self):
        scores = {
            "loss" : self.score[0],
            "accuracy" : self.score[1]
        }
        return save_json(path = Path('scores.json'), data = scores)

In [70]:
try:
    config = ConfigurationManager()
    validation_config = config.get_evaluation_config()
    evaluation = ModelEvaluation(validation_config)
    evaluation.evaluate()
    evaluation.save_scores()
except Exception as e:
    raise e

Found 7498 images belonging to 2 classes.
469/469 [==============================] - 69s 146ms/step - loss: 1.5619 - accuracy: 0.8665
